In [ ]:
# this file will just be data processing loool 
# maybe put info into csv file (song title,song artist,mood1,mood2,...)

# for every file in millionsongsubset, we need to get the songs
# for every song, we need to webscrape allmusic to find the moods

# can download a subset by using wget and a url from the million song database website

In [1]:
# imports?
# pip install tables
# pip install pyquery
# pip install fuzzywuzzy
# pip install selenium
import tarfile
import os
import hdf5_getters
import numpy as np
import csv
import allmusic_scraping
from bs4 import BeautifulSoup
import requests
from pyquery import PyQuery as pq
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from seleniumbase import Driver
from selenium.common.exceptions import TimeoutException

/opt/conda/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# extract msd data
def extract_tar_gz(file_path, extract_path):
    # Open the tar.gz file
    with tarfile.open(file_path, 'r:gz') as tar:
        # Extract all files
        tar.extractall(path=extract_path)

# Example usage:
tar_gz_file = 'millionsongsubset.tar.gz'
extract_directory = 'millionsongsubset'

# Make sure the extract directory exists
os.makedirs(extract_directory, exist_ok=True)

# Extract the tar.gz file
extract_tar_gz(tar_gz_file, extract_directory)

# after extracting the file, flatten the directories with this terminal call:
# $ find coms4995-appliedcv/millionsongsubset/MillionSongSubset -type f -exec mv -i '{}' ./coms4995-appliedcv/millionsongsubset \;

# now remove the folder with the redundant files
# $ cd coms4995-appliedcv/millionsongsubset/
# $ rm -r MillionSongSubset/

In [2]:
# write to a csv file (song title,song artist,mood1,mood2,...)
def read_moods(file_path):
    with open(file_path, 'r') as file:
        moods = []
        moods_dict = {}
        count = 0
        for line in file:
            m = line.strip().lower()
            moods.append(m)
            moods_dict[m] = count
            count += 1
        return moods, moods_dict
        #return [line.strip() for line in file]

# moods.txt from https://github.com/fdlm/listening-moods/blob/master/data/moods.txt
moods_file = 'moods.txt'

# Read moods from file
moods, moods_dict = read_moods(moods_file)

# Headers contain song title, song artist, and moods
headers = ['song_title', 'song_artist'] + moods

In [12]:
x = "hello"
x[:x.index('l')]

'he'

In [ ]:
# Directory containing the .h5 files
directory = './millionsongsubset'
data = []
num_moods = len(moods)

# List all files in the directory
for filename in os.listdir(directory):
    # Check if the file is an .h5 file
    if filename.endswith('.h5'):
        # Process the .h5 file
        file_path = os.path.join(directory, filename)
        # print(f"Processing file: {file_path}")
        # process the .h5 file
        h5 = hdf5_getters.open_h5_file_read(file_path)
        num_songs = hdf5_getters.get_num_songs(h5)
        
        for i in range(num_songs):
            song_data = []
            song_title = hdf5_getters.get_title(h5,i)
            song_title = song_title.decode('utf-8').lower()
            
            try:
                parentheses = song_title.index('(')
            except ValueError:
                parentheses = -1
            
            if parentheses != -1:
                song_title = song_title[:parentheses]
                
            song_artist = hdf5_getters.get_artist_name(h5,i)
            song_artist = song_artist.decode('utf-8').lower()
            
            song_data.append(song_title)
            song_data.append(song_artist)
            
            song_data += [0]*num_moods
            
            # song_search_matching
            # https://github.com/jack-arms/allmusic-python/blob/master/allmusic.py
            # query: chart_song.title + ' ' + main_artist
            # print(song_title)
            # print(song_artist)
            allmusic_song = allmusic_scraping.song_search(song_title + ' ' + song_artist, 3)
            # print(allmusic_song)
            # find mood for the song
            # check if there was an error during the search
            if isinstance(allmusic_song, dict) and 'error' in allmusic_song:
                print("Error:", allmusic_song['error'])
                continue # throw error?

            # check if any search results were found
            if not allmusic_song:
                print("No search results found for the given query.")
                continue # throw error?

            top_result = allmusic_song[0]

            # get html url
            html_page_url = top_result['title']['url'] + "#moodsThemes"

            # fetch html
            if html_page_url:
                print(html_page_url)
                html_page = requests.get(html_page_url)
                html_page = requests.get(html_page_url, headers={
                    'Host': 'www.allmusic.com',
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
                    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
                    'Accept-Encoding': 'gzip, deflate, sdch',
                    'Accept-Language': 'en-US,en;q=0.8'
                })
                driver = Driver(browser="chrome", headless=False)
                
                driver.get(html_page_url)
                
                try:
                    try:
                        scraped_moods = WebDriverWait(driver, 2).until(
                            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div#moodsGrid > div > a'))
                        )
                    except TimeoutException:
                        print("didn't find moods")
                        continue
                    
                    mood_words = [m.text[:m.text.index(' ')].lower() for m in scraped_moods]
                    print(mood_words)
                    # go through mood_words and one hot encode it in data
                    for mood in mood_words:
                        if mood in moods_dict:
                            song_data[moods_dict[mood]+2] = 1
                    
                finally:
                    driver.quit()

#                 if html_page.status_code == 200:
#                     soup = BeautifulSoup(html_page.text, 'html.parser')
#                     # iframes = soup.find_all('iframe')
#                     # # print(html_page.text)
#                     mood_grid_div = soup.find_all('div', {'id':'moodsGrid'})
#                     # # print(mood_grid_div)
                    
#                     # Use PyQuery to parse the HTML content
#                     # d = pq(html_page.text)
#                     # Find the div with id 'moodsGrid'
#                     # mood_grid_div = d('div#moodsGrid')

#                     # mood_grid_div = d('div#moodsGrid')
#                     # mood_grid_div = d.xpath('//div[@id="moodsGrid"]')
#                     # mood_grid_div = d('/html/body/div[1]/div/section/div[2]/div[8]/div[2]/div[1]')
#                     print(type(mood_grid_div))
#                     print(mood_grid_div)
#                     if mood_grid_div:
#                         # find all 'a' tags within the 'moodGrid' div
#                         a_tags = mood_grid_div.find('a')
#                         print("a_tags", a_tags)
#                         # get text from each 'a' tag
#                         mood_words = [a_tag.text for a_tag in a_tags]
#                         print(mood_words)
#                         # go through mood_words and one hot encode it in data
#                         for mood in mood_words:
#                             song_data[moods_dict[mood]+2] = 1
#                     else:
#                         continue # no moods found.. don't add to data?
#                 else:
#                     print("Failed to fetch HTML page:", html_page.status_code)
            else:
                print("No URL found for the top search result.")
            
            data.append(song_data)
            
        h5.close()
        

inside scraper dirty dancer   michael white
https://www.allmusic.com/song/dirty-dancer-mt0004502108#moodsThemes
didn't find moods
inside scraper everywhere  michelle tumes
https://www.allmusic.com/song/everywhere-mt0005074330#moodsThemes
didn't find moods
inside scraper finding my way stanley clarke & george duke
https://www.allmusic.com/song/finding-my-way-mt0053351035#moodsThemes
didn't find moods
inside scraper de tha mai edo domna kountouri
https://www.allmusic.com/song/de-tha-mai-edo-mt0016871171#moodsThemes
didn't find moods
inside scraper sally can't dance lou reed
https://www.allmusic.com/song/sally-cant-dance-mt0006125722#moodsThemes
['brooding', 'confident', 'greasy', 'gritty', 'gutsy', 'nocturnal', 'passionate', 'reflective', 'sophisticated', 'street-smart', 'swaggering', 'lively']
inside scraper my fofo   fat joe
https://www.allmusic.com/song/my-fofo-mt0039479117#moodsThemes
didn't find moods
inside scraper it must be love  rickie lee jones
https://www.allmusic.com/song/it-

KeyError: 'lazy'

In [7]:
moods_dict

{'acerbic': 0,
 'aggressive': 1,
 'agreeable': 2,
 'ambitious': 3,
 'amiable/good-natured': 4,
 'angry': 5,
 'angst-ridden': 6,
 'anguished/distraught': 7,
 'animated': 8,
 'atmospheric': 9,
 'austere': 10,
 'autumnal': 11,
 'belligerent': 12,
 'bitter': 13,
 'bittersweet': 14,
 'bleak': 15,
 'boisterous': 16,
 'brash': 17,
 'brassy': 18,
 'bravado': 19,
 'bright': 20,
 'brittle': 21,
 'brooding': 22,
 'calm/peaceful': 23,
 'campy': 24,
 'carefree': 25,
 'cathartic': 26,
 'celebratory': 27,
 'cerebral': 28,
 'cheerful': 29,
 'circular': 30,
 'clinical': 31,
 'cold': 32,
 'complex': 33,
 'confident': 34,
 'confrontational': 35,
 'crunchy': 36,
 'cynical/sarcastic': 37,
 'delicate': 38,
 'detached': 39,
 'difficult': 40,
 'dramatic': 41,
 'dreamy': 42,
 'driving': 43,
 'druggy': 44,
 'earnest': 45,
 'earthy': 46,
 'eccentric': 47,
 'eerie': 48,
 'effervescent': 49,
 'elaborate': 50,
 'elegant': 51,
 'energetic': 52,
 'enigmatic': 53,
 'epic': 54,
 'ethereal': 55,
 'exciting': 56,
 'exube

In [8]:
def write_csv(file_path, data, headers):
    with open(file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(headers)
        for row in data:
            writer.writerow(row)

In [4]:
data

[["sally can't dance",
  'lou reed',
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 ['it must be love ',


In [ ]:
song_csv = 'song_moods.csv'

# Write to CSV
write_csv(song_csv, data, headers)

In [ ]:
# go through all the files in /millionsongsubset
# for each file, find the songs in allmusic.com (id="moodsGrid")

In [ ]:
# for each song in msd, webscrape allmusic for moods

In [ ]:
# find spotify id for song

In [ ]:
# store in dictionary {mood: [spotify id]}